In [1]:
from pygraylog.graylogapi import GraylogAPI

In [2]:
graylog_con={
        "user":"merger",
        "password":"pc@pM8rger",
        "url":"http://graylog-ui.roamability.com:9000/api"
    }

In [3]:
api = GraylogAPI(
    graylog_con["url"],
    graylog_con["user"], 
    graylog_con["password"])

In [4]:
import datetime
#format like 2017-03-06T16:28
dateFrom = datetime.datetime.strptime('2018-01-12T05:00', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-01-12T12:00', '%Y-%m-%dT%H:%M')
query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:"a2:2a:0c:40"'

In [6]:
res=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo)

In [9]:
res.type

AttributeError: 'unicode' object has no attribute 'type'